# Imports & Global Arguments

In [1]:
import os
import json

In [23]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /home/maxim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/maxim/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [25]:
PROJECT_ROOT_PATH = os.path.expanduser("~")
DRYAD_DATASET_PATH = os.path.join(PROJECT_ROOT_PATH, "assets", "rsos_dream_data.tsv")
DREAM_DATASET_PATH = os.path.join(PROJECT_ROOT_PATH, "assets", "dream_train.json")
MAX_TEXT_INPUT_LENGTH = 5000
MAX_TEXT_OUTPUT_LENGTH = 5000
FLAN_T5_SMALL_NAME = "google/flan-t5-small"

# Datasets Init

## Freud Interpret

In [17]:
freud_dataset_raw = {
  "I was walking across a narrow bridge high above a canyon when suddenly the ground beneath me gave way. I felt myself plummeting into the abyss, unable to grab onto anything to stop my fall.": "This dream represents fears of losing control in a critical situation. The unstable bridge could symbolize fragile confidence or an uncertain path in waking life, while the fall reflects anxieties about failing to maintain stability or achieve a desired outcome.",
  "I was climbing a steep mountain, desperately trying to reach the top, but my grip slipped, and I tumbled down, feeling a sense of dread and helplessness as I fell further and further away from the summit.": "This dream symbolizes fear of failure or setbacks in achieving ambitious goals. The mountain represents aspirations, while the fall may indicate a perceived inability to meet expectations or maintain progress in waking life.",
  "I was flying through the sky, feeling free and weightless, but suddenly I began to fall. The ground came rushing toward me, and no matter how much I tried, I couldn’t stop myself from crashing.": "This dream reflects a fear of losing control after experiencing success or freedom. The fall represents anxieties about an imminent failure or the inability to sustain the elevated state of achievement in waking life.",
  "I was standing on a crowded train platform, waiting for my train, when I suddenly realized I wasn’t wearing any clothes. Everyone around me started staring and whispering, and I felt frozen, unable to cover myself or escape.": "This dream symbolizes feelings of vulnerability and being judged by others. The crowded platform represents a social situation where the dreamer may feel exposed or unprepared, reflecting anxieties about public perception or criticism in waking life.",
  "I was giving a presentation in front of my colleagues, feeling confident, until I looked down and saw that I was completely naked. The room erupted in laughter, and I felt a wave of shame and panic.": "This dream reflects fear of exposure or inadequacy in a professional setting. The nakedness signifies a fear of being unprepared or having personal flaws revealed in front of others, especially in a situation requiring confidence or competence.",
  "I was walking through a busy city street, surrounded by strangers, when I noticed I had no clothes on. No one seemed to care or notice, but I felt deeply embarrassed and tried to hide behind objects as I moved.": "This dream suggests feelings of self-consciousness and a heightened sense of exposure, even when others may not be paying attention. It reflects an internal struggle with insecurity and the dreamer’s fear of being noticed or judged.",
  "I found myself soaring over a dense forest, the treetops stretching endlessly below me. The wind rushed past my face, and I felt an incredible sense of freedom, as if nothing in the world could hold me back.": "This dream symbolizes a deep desire to escape from life’s constraints or responsibilities. The act of flying over the forest suggests a longing to rise above challenges or break free from situations that feel overwhelming or restrictive in waking life.",
  "I was flying above a vast ocean, skimming the waves with my fingertips. The water sparkled under the sunlight, and I felt light and unburdened, as if all my worries had been left behind on the shore.": "This dream represents a yearning for emotional freedom and relief from stress. The ocean often symbolizes the subconscious, and flying over it suggests the dreamer’s wish to transcend deep-seated fears or emotional burdens, seeking liberation and peace.",
  "I was running through a crowded city, feeling trapped by the chaos, when suddenly I lifted off the ground and flew above the buildings. From above, everything seemed calm, and I felt a sense of control and clarity I hadn’t felt in a long time.": "This dream reflects a desire to escape the pressures of daily life. The crowded city symbolizes stress or societal demands, while flying represents the dreamer’s wish to gain a new perspective and freedom from those constraints, reclaiming a sense of control.",
  "I was talking to a friend when I suddenly felt a strange looseness in my mouth. One by one, my teeth began falling out into my hands. I tried to speak, but my words came out garbled, and my friend just stared at me in shock.": "This dream represents anxiety about communication—the inability to speak properly may symbolize a fear of saying the wrong thing or being misunderstood in waking life. It may also reflect social insecurities, particularly about how others perceive the dreamer.",
  "I was looking in the mirror, brushing my teeth, when I noticed they were crumbling like sand. No matter how much I tried to hold them together, they kept falling apart, leaving me feeling helpless and horrified.": "This dream is associated with worries about appearance and self-image. The crumbling teeth may symbolize a fear of aging, losing attractiveness, or feeling powerless over changes in one’s life. It could also reflect a loss of confidence or fear of personal deterioration.",
  "I was in an important meeting at work, speaking confidently, when I suddenly felt my teeth becoming loose. As I kept talking, they started falling out onto the table. I tried to cover my mouth, but everyone had already noticed": "This dream reflects performance anxiety and fear of public embarrassment. It suggests the dreamer may feel insecure about their professional abilities or worry about being judged by colleagues. Losing teeth in front of others can symbolize a fear of losing credibility or respect in a social or work-related environment.",
  "I dreamed that I was attending my own funeral, watching from a distance as people I knew gathered to say goodbye. Strangely, I didn’t feel scared—just calm, as if I were preparing to leave behind an old version of myself": "This dream symbolizes a major life transition or personal transformation. Attending one’s own funeral often represents letting go of the past, shedding old habits, or embracing a new phase in life. It may reflect a psychological rebirth, where the dreamer is moving beyond an old identity or way of living.",
  "I dreamed that a close friend passed away. I was devastated, but then I saw them again, alive and different somehow—like they had changed into someone new.": "This dream is not necessarily about literal death but rather about change. The death of a friend in a dream may indicate that the relationship itself is evolving, or that the dreamer sees a change in the friend’s personality or role in their life. It could also represent fears of losing connection with them.",
  "I was walking through an old house when I stumbled upon a grave with my childhood name on it. I stood there, realizing that the person I used to be no longer existed.": "This dream suggests a symbolic death of the past self. The grave with a childhood name represents growing up, maturing, or leaving behind past behaviors. It may indicate a moment of self-realization where the dreamer acknowledges personal growth or a shift in identity.",
  "I was sitting in a huge exam hall, staring at a test paper filled with questions I had never seen before. No matter how hard I tried to answer, my mind went blank. The clock was ticking loudly, and I could feel panic rising as time slipped away.": "This dream reflects anxiety about performance and judgment. The inability to answer the questions suggests the dreamer may feel unprepared or inadequate in some aspect of their waking life—whether at work, in relationships, or in personal expectations.",
  "I arrived at school, realizing I had completely forgotten about an important final exam. I frantically searched for a pen, but every time I found one, it wouldn’t write. The teacher glared at me as I struggled, feeling helpless and embarrassed.": "This dream represents fear of failure and a lack of control. The forgotten exam may symbolize unconscious self-doubt, while the inability to write can reflect a fear of being powerless or incapable in a real-life situation where the dreamer feels pressured to perform well.",
  "I was taking a test in a subject I had never studied, but strangely, I felt calm. As I flipped through the pages, I realized the questions were about my own life—decisions I had made, relationships I had built. The answers were not right or wrong, only reflections of my past choices.": "This dream suggests self-evaluation and introspection rather than external pressure. Instead of fearing judgment, the dreamer may be going through a phase of self-reflection, questioning past decisions and considering how they have shaped their present reality.",
  "I was running through a dark forest, my heart pounding as something unseen chased me. No matter how fast I ran, I could hear footsteps closing in behind me. I tried to scream, but no sound came out, and I felt completely powerless.": "This dream represents avoidance of a deep-seated fear or unresolved issue in waking life. The unknown pursuer could symbolize anxiety, guilt, or a problem the dreamer is trying to escape rather than confront. The inability to scream suggests feelings of helplessness or repression.",
  "A masked figure was chasing me through a city. I darted through alleyways and hid behind cars, but no matter where I went, they always found me. Just as they reached out to grab me, I woke up in a panic.": "This dream may reflect stress or pressure from an external force—such as work, relationships, or responsibilities that feel overwhelming. The masked figure could symbolize an unknown fear or a situation the dreamer is trying to avoid, but it keeps resurfacing.",
  "I was being chased by a wild animal, running as fast as I could. Just before it caught me, I turned around and realized the animal was no longer there—I had been running from nothing.": "This dream suggests that the dreamer may be running from an imagined or exaggerated fear. It could indicate unnecessary worry or avoidance of an issue that is not as dangerous as it seems. Facing the fear directly may bring relief in waking life.",
  "I was standing on the edge of a skyscraper, looking down at the city below. Suddenly, the ground beneath me crumbled, and I began to fall. My arms flailed, but there was nothing to hold onto. As I plummeted, my stomach dropped, and I felt completely powerless, knowing there was no way to stop myself from hitting the ground.": "This dream symbolizes a fear of losing control in an important aspect of life—whether in career, relationships, or personal stability. The high place represents ambition or success, while the fall reflects anxieties about failing or losing everything. It may also indicate a fear of uncertainty or instability, suggesting that the dreamer feels overwhelmed by circumstances beyond their control.",
  "I was walking into a classroom filled with students when I suddenly realized I wasn’t wearing any clothes. My heart pounded as I tried to cover myself with my hands, but everyone was staring at me. Some laughed, while others just looked at me with confusion. I wanted to run, but my feet wouldn’t move.": "This dream reflects anxiety about judgment and self-image, particularly in academic or social settings. The classroom suggests a fear of intellectual or personal inadequacy, while the inability to escape may indicate feeling trapped in a situation where the dreamer fears being exposed as unprepared or incapable.",
  "I wasl at a wedding, surrounded by elegantly dressed guests, when I looked down and saw that I was completely naked. No one seemed to notice at first, but as I tried to act normal, whispers and shocked glances spread through the crowd. My face burned with embarrassment as I searched for something to cover myself.": "This dream suggests a fear of not meeting social expectations or feeling out of place in formal or high-pressure environments. Weddings often symbolize life transitions, so the nakedness could indicate anxiety about a major change in the dreamer’s personal or social life, such as commitment, relationships, or status.",
  "I was giving a speech at a large event, feeling confident at first, until I noticed people in the audience giggling. When I looked down, I realized I was wearing nothing. I tried to continue speaking, but the more I spoke, the louder the laughter became, until I could barely hear myself.": "This dream represents performance anxiety and fear of public humiliation. The act of giving a speech symbolizes self-expression or professional exposure, while the laughter from the audience suggests a deep-seated fear of criticism or being unprepared in a crucial moment. It may indicate self-doubt in professional or social settings.",
  "I was running through a field when suddenly my feet lifted off the ground. I soared higher and higher, feeling weightless as I glided over the landscape. The sky was bright and endless, and I felt an incredible sense of freedom, as if I had escaped all my worries.": "This dream symbolizes a deep longing for freedom and escape from life’s burdens. The effortless flight represents a desire to break free from stress, responsibilities, or emotional constraints. It may indicate a need to step back from daily pressures and regain a sense of control.",
  "I was trapped inside a dark, windowless room, struggling to find a way out. Just as I was about to give up, I felt my body become lighter, and I floated toward the ceiling. I passed through the walls and into the open sky, leaving the darkness behind.": "This dream reflects a need for escape from a restrictive situation. The dark room symbolizes feeling trapped—either emotionally, mentally, or in a real-life circumstance. Flying away represents breaking free from limitations, whether they be personal, professional, or psychological.",
  "I was flying over an ocean, the waves shimmering below me as the wind carried me effortlessly. But as I tried to go higher, something invisible pulled me back down, making it harder and harder to stay in the air.": "This dream represents an internal conflict between the desire for freedom and external forces holding the dreamer back. The ocean can symbolize the subconscious mind or emotional depth, while the struggle to fly higher suggests real-life obstacles—such as responsibilities, fears, or doubts—that prevent full liberation.",
  "I was flying over a city at night, watching the streets and buildings below. I felt powerful, like I could go anywhere, but I realized I was alone. There was no one else flying with me, and I suddenly felt disconnected from the world below.": "This dream suggests a sense of independence and control but also isolation. The ability to fly symbolizes personal empowerment and self-sufficiency, while the loneliness may indicate feeling detached from others due to personal ambitions or life changes. It may suggest a need to balance independence with connection.",
  "I was in the middle of a heated argument when, suddenly, I pushed off the ground and flew away. As I soared above the situation, the shouting faded, and I felt an overwhelming sense of relief.": "This dream symbolizes a strong desire to avoid confrontation or escape emotional conflicts. Flying away represents removing oneself from a stressful or negative situation rather than facing it directly. It may indicate a tendency to disengage from problems instead of resolving them.",
  "I was laughing with a group of people when I suddenly felt something strange in my mouth. I reached in and pulled out a loose tooth, but as soon as I did, more teeth started falling out one by one. I tried to hide it, but everyone noticed and looked horrified.": "This dream symbolizes anxiety about social perception and self-image. The dreamer may feel insecure about how others see them, particularly in social or professional settings. It could also indicate a fear of embarrassment or saying the wrong thing in an important situation.",
  "I was chewing on something when I suddenly realized my teeth were crumbling into tiny pieces. No matter how much I tried to spit them out, more and more broke apart, until my entire mouth felt empty and powerless.": "This dream reflects a feeling of losing control or power in communication. The act of chewing suggests an attempt to process or handle a situation, but the crumbling teeth indicate a fear of being unable to express oneself effectively or feeling weak in a certain aspect of life.",
  "I looked in the mirror and noticed my teeth were rotting and falling out. I tried to push them back in, but they kept slipping through my fingers. My reflection looked older and unfamiliar, and I felt a wave of panic.": "This dream represents fears related to aging, self-worth, and appearance. The mirror and the decaying teeth may indicate anxiety about physical deterioration, attractiveness, or self-confidence. It could also symbolize feeling unable to stop an inevitable change in life.",
  "I was about to give a speech in front of a large audience when my teeth started to loosen. As I spoke, they fell out onto the podium one by one. I tried to keep talking, but my voice became distorted, and people in the audience began whispering and laughing.": "This dream suggests a fear of public speaking, performance anxiety, or communication failure. Losing teeth while speaking may reflect a deep concern about being judged, misunderstood, or saying something embarrassing. It could also indicate a fear of losing authority or credibility in a situation where the dreamer wants to be taken seriously.",
  "I was on a date with someone I really liked, feeling confident, when suddenly I felt my teeth becoming loose. I tried to keep smiling and talking, but they started falling out into my hand. My date looked disgusted, and I wanted to disappear.": "This dream represents insecurities in romantic or personal relationships. It may indicate a fear of rejection, not being good enough, or feeling exposed in front of someone important. The teeth falling out symbolize a loss of confidence and control in an intimate or vulnerable situation.",
  "I dreamed that my childhood home was being demolished, and as I stood there watching, I saw my younger self inside, waving goodbye before disappearing in the dust. I felt a strange mix of sadness and relief.": "This dream symbolizes the end of a significant life phase and personal transformation. The destruction of the childhood home represents leaving behind old identities, memories, or past attachments, while the younger self waving suggests acknowledging growth and moving forward.",
  "I was walking through a cemetery when I found a gravestone with my name on it. Instead of feeling afraid, I felt peaceful, as if I was ready to move on to something new. Flowers were growing around the grave, and the air felt fresh and light.": "This dream represents acceptance of change and renewal. Seeing one’s own gravestone can symbolize the end of an old version of the self—whether it’s a mindset, habit, or life stage. The peaceful feeling and flowers suggest growth, transformation, and a positive transition into something new.",
  "I dreamed that I was on a sinking ship, struggling to stay afloat. As the water rose, I panicked, but then I suddenly stopped fighting and let myself sink. Instead of drowning, I found myself in a completely new place, as if I had been reborn.": "This dream symbolizes letting go of control and embracing change. The sinking ship represents a part of life that is ending or no longer serving the dreamer, while the transition to a new place suggests a transformation or a fresh start in waking life.",
  "I was attending a funeral for someone I didn’t know, but when I looked inside the casket, I saw a version of myself lying there. Strangely, I felt no fear—just a quiet understanding that something in my life had come to an end.": "This dream suggests a deep personal transformation or shedding of an old identity. Seeing oneself in a casket can represent leaving behind outdated beliefs, behaviors, or past experiences. The calmness in the dream indicates that the dreamer is subconsciously ready for this transition.",
  "I dreamed that a close friend died, and I was devastated. But later in the dream, I saw them again—alive, but different somehow. They looked older, wiser, and distant, as if they had moved on from our friendship.": "This dream may not be about literal death but rather a changing relationship. The death of a friend in a dream can symbolize the evolution or fading of a connection. Seeing them “reborn” as someone different suggests subconscious recognition that the dynamic of the relationship is shifting or that the dreamer is growing apart from them.",
  "I arrived at an exam hall, but when I looked at the paper, the questions were in a language I didn’t understand. The professor stood over me, waiting for me to write something, but I felt frozen, unable to make sense of a single word.": "This dream represents a fear of being unprepared or feeling out of place in a high-pressure situation. The unfamiliar language symbolizes a lack of confidence or knowledge in a specific area of waking life, possibly in a job, relationship, or personal expectation.",
  "I was taking a math test, but no matter how hard I tried, I couldn’t solve a single problem. My pencil kept breaking, the numbers blurred on the page, and the more I panicked, the faster the clock ticked down.": "This dream reflects performance anxiety and a fear of failure. Math often represents logic and problem-solving, so the inability to answer questions may indicate stress about decision-making or feeling mentally overwhelmed in real life.",
  "I walked into a classroom only to realize I had missed the entire semester and now had to take the final exam. My classmates looked at me with pity as I flipped through the test paper, completely lost.": "This dream suggests feelings of inadequacy, imposter syndrome, or guilt about not meeting expectations. It may stem from a fear of being exposed as unprepared or lacking the necessary skills in an important aspect of life.",
  "I was taking an exam, but instead of using a pen, I had to answer aloud in front of a huge audience. Every time I got a question wrong, the crowd would laugh, and I felt smaller and smaller, as if I might disappear.": "This dream symbolizes fear of public embarrassment or social judgment. The exam represents self-evaluation, while the audience reflects external pressures or criticism in real life—perhaps from work, family, or peers.",
  "I finally finished an impossible exam and felt relieved—only to have the teacher hand me another test, even harder than the last. No matter how many I completed, they just kept coming, and I realized I would never be done.": "This dream reflects a feeling of constant pressure or never being ‘good enough’. It suggests that the dreamer may feel like no matter how much they accomplish, new challenges always arise, leading to exhaustion and stress.",
  "I was running through an empty shopping mall, chased by a shadowy figure. No matter which store I ran into or how many turns I took, the figure was always just behind me. When I finally turned around to confront it, I woke up.": "This dream suggests a subconscious avoidance of a fear or unresolved issue. The shadowy figure may represent a hidden anxiety, past mistake, or responsibility that the dreamer has been trying to ignore. The inability to escape could indicate a need to finally confront the problem.",
  "I was being chased by a pack of wild dogs through a narrow alley. Their growls got louder as they got closer, and I could feel my heart pounding. Just as they were about to catch me, I found a door and slammed it shut behind me.": "This dream may symbolize overwhelming stress, pressure, or aggression in waking life. The dogs could represent hostile people, demanding responsibilities, or personal fears that feel uncontrollable. Closing the door suggests a temporary escape, but the unresolved fear may still be present.",
  "I was walking home when I noticed someone following me. No matter how fast I walked, they stayed the same distance behind me. I never saw their face, but I knew they were always there, just out of sight.": "This dream represents a lingering fear or unresolved issue that the dreamer cannot escape. The faceless pursuer may symbolize a hidden worry, guilt, or responsibility that follows the dreamer in waking life, even when they try to ignore it.",
  "I was running from a giant wave crashing behind me. No matter how fast I ran, the water kept getting closer, and I knew I couldn’t outrun it. Just before it hit me, I woke up.": "This dream suggests feeling overwhelmed by emotions, stress, or an unavoidable life event. The wave could symbolize suppressed feelings, anxiety, or external pressures that the dreamer is trying to avoid but knows they must eventually face.",
  "I was being chased through a dark forest by a version of myself—only this version looked angry and determined. Every time I tried to hide, it found me. It felt like I was running from my own reflection.": "This dream symbolizes inner conflict and self-avoidance. Being chased by oneself may indicate denying an aspect of one’s personality, emotions, or past actions. It suggests that the dreamer needs to confront their own thoughts, feelings, or decisions rather than running from them."
}
freud_df = pd.DataFrame(list(freud_dataset_raw.items()), columns=["dream_description", "dream_interpretation"])
print(freud_df.head(10))
freud_train_df = freud_df.sample(40, random_state=42)
freud_test_df = freud_df.drop(freud_train_df.index)


                                   dream_description  \
0  I was walking across a narrow bridge high abov...   
1  I was climbing a steep mountain, desperately t...   
2  I was flying through the sky, feeling free and...   
3  I was standing on a crowded train platform, wa...   
4  I was giving a presentation in front of my col...   
5  I was walking through a busy city street, surr...   
6  I found myself soaring over a dense forest, th...   
7  I was flying above a vast ocean, skimming the ...   
8  I was running through a crowded city, feeling ...   
9  I was talking to a friend when I suddenly felt...   

                                dream_interpretation  
0  This dream represents fears of losing control ...  
1  This dream symbolizes fear of failure or setba...  
2  This dream reflects a fear of losing control a...  
3  This dream symbolizes feelings of vulnerabilit...  
4  This dream reflects fear of exposure or inadeq...  
5  This dream suggests feelings of self-conscious... 

### What is Freud Interpret?
This is a manually generated dataset of 50 dreams. They are associated with the common Freud interpretations:
* Dreams about falling: Often related to fears of losing control or failing.
* Dreams about being naked in public: Can represent feelings of vulnerability or exposure.
* Dreams about flying: May symbolize a desire for freedom or escape from constraints.
* Dreams about teeth falling out: Could represent anxiety about appearance or communication.
* Dreams about death: Often symbolic of change or transition rather than literal death.
* Dreams about exams or tests: May reflect feelings of being judged or evaluated in waking life.
* Dreams about being chased: Could represent avoiding a problem or fear in waking life.

The dataset has no additional data. This is intended to ensure the research of how well models trained on the following datasets cope with unseen, "raw" dreams and custom interpretations, i.e. by Freud rules.

## Dryad

In [7]:
dryad_df = pd.read_csv(DRYAD_DATASET_PATH, sep='\t')
dryad_df = dryad_df.head(10) #TODO: remove
print(dryad_df)

   dream_id dreamer               description dream_date dream_language  \
0         1    alta  Alta: a detailed dreamer       1957             en   
1         2    alta  Alta: a detailed dreamer  8/11/1967             en   
2         3    alta  Alta: a detailed dreamer   8/1/1985             en   
3         4    alta  Alta: a detailed dreamer      1985?             en   
4         5    alta  Alta: a detailed dreamer      1985?             en   
5         6    alta  Alta: a detailed dreamer      1985?             en   
6         7    alta  Alta: a detailed dreamer      1985?             en   
7         8    alta  Alta: a detailed dreamer      1985?             en   
8         9    alta  Alta: a detailed dreamer      1985?             en   
9        10    alta  Alta: a detailed dreamer      1985?             en   

                                          text_dream  \
0  The one at the Meads's house, where it's bigge...   
1  I'm at a family reunion in a large fine house ...   
2  I w

### What is Dryad dataset?
["Our Dreams, Our Selves: Automatic Interpretation of Dream Reports"](https://datadryad.org/stash/dataset/doi:10.5061/dryad.qbzkh18fr) dataset comprises over 20,000 dream reports sourced from https://dreambank.net. It is well-suited for Natural Language Processing tasks, particularly in the realm of automated text analysis and classification:
* Identify Characters: Detect and classify entities within dream narratives.
* Analyze Interactions: Determine the nature of interactions (e.g., friendly, aggressive, sexual) between characters.
* Assess Emotional Tone: Evaluate the emotional content and sentiment expressed in the text.

The automated text analysis and classification provides a "logical bridge" to Freud dream analysis, which can be outlined as follows:
1. Train the model on Dryad dataset.
2. "Translate" dreams from "Freud Interpret" dataset to Dryad encoding format.
3. Further train the model with the combined input of 40 "Freud Interpret" dream description, associated Dryad encoding format and expected output of Freud interpretation.
4. Validate the model on the rest of the 10 "Freud Interpret" dreams

## DREAM

In [8]:
dream_dataset_raw = None
with open(DREAM_DATASET_PATH, "r") as a_file:
  dream_dataset_raw = json.load(a_file)
dream_df_rows_raw = []
for an_item in dream_dataset_raw:
  a_dialogue = " ".join(an_item[0])
  for a_question_item in an_item[1]:
    a_question = a_question_item["question"]
    multiple_choices = a_question_item["choice"]
    an_answer = a_question_item["answer"]
    dream_df_rows_raw.append({
      "dialogue": a_dialogue,
      "question": a_question,
      "multiple_choice": multiple_choices,
      "correct_answer": an_answer
    })
dream_df = pd.DataFrame(dream_df_rows_raw)
dream_df = dream_df.head(10) #TODO: remove
print(dream_df)

                                            dialogue  \
0  M: I am considering dropping my dancing class....   
1  W: Well, I'm afraid my cooking isn't to your t...   
2  W: Well, I'm afraid my cooking isn't to your t...   
3  M: Look at the girl on the bike! F: Oh, yes sh...   
4  M: Can you accompany me while I am singing. W:...   
5  W: I can't figure out what's wrong with my wat...   
6  W: Hello, this is TBC Television Studios. How ...   
7  W: Hello, this is TBC Television Studios. How ...   
8  M: I think it's going to rain. W: I guess so. ...   
9  W: Front desk. Can I help you? M: Yeah, hi, th...   

                                          question  \
0          What does the man suggest the woman do?   
1  What does the man think of the woman's cooking?   
2           What does the woman ask the man to do?   
3                       Where are the two persons?   
4          What does the man want the woman to do?   
5                          What does the man mean?   
6    

### What is DREAM dataset?
[DREAM](https://dataset.org/dream/) is a multiple-choice Dialogue-based REAding comprehension exaMination dataset. It's data can be examined at [official repository](https://github.com/nlpdata/dream/tree/master/data).

Dialogue presents an outstanding dataset trait, which presents a significant challenge for nowadays LLMs. The prediction of correct asnwer inherently requires from a model to "reason" beyond the standard approach of `next 1 or 2 words prediction` even when concept of "attention" is utilized.

Which is what makes this dataset a suitable train source for our task at hand. Mapping dreams to Freud interpretations is inherently more complicated than a naive classification combined with prediction of the next `N` words. There is a requirement to "reason" about interpretation with a dream's summary "in mind".

# Models Choice

## Flan-T5-Small
[Google's Flan-T5-Small](https://huggingface.co/google/flan-t5-small) is the first model choice, which will be trained on the "Dryad" dataset. This is a "light" version of [Google's T5-large](https://huggingface.co/google-t5/t5-large).

The motivation is T5 specializing at [Transfer learning](https://en.wikipedia.org/wiki/Transfer_learning). According to the [official accompanying paper](https://jmlr.org/papers/volume21/20-074/20-074.pdf), we assume that the model is expected to sort of "gracefully translate" a dream's summary to a Freud interpretation. The underlying principle in T5 is the treatment of every text processing problem as a "text-to-text" problem. Which suits our task at hand.

Translation is not the only model's strong trait. It also specializes at abstract summariztion, unlike its counterparts from BERT family.

The secondary reasoning for choosing Flan-T5-Small is our computational resources. The research is conducted on Macbook Pro M1 machine, which has modest graphical processing capabilities.

## DistilBERT
[DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert) is the second model choice, which will be trained on the "DREAM" dataset. This is a "light" version of [BERT](https://arxiv.org/abs/1810.04805).

The motivation behind the light version is computational environment constraints, similar to motivation behind Flan-T5-Small.

More important though, is the fact that "BERT" is sort of an alternative strategy to "T5-large". It is designed to pretrain deep bidirectional representations. This makes it a perfect candidate for question answering and language inference with just an additional single layer during fine-tuning. In our case, "DREAM" dataset will be such a layer. Especially, given its dialogue-like nature of data.

# Research question
We would like to measure models' performance on the Freud dream interpretation task with the following metrics:
* BLEU
* ROUGE
* Perplexity
* BERT score

Our expected observations are two-fold.  
First, both models are expected to perform poorly on the Freud dataset. This is because of an inherent low probability that their training involved such a specific task. However, there is an option that Freud interpretation system itself was actually covered, thanks to the vast data volume used during both models' training.  
Still, we expect a poor performance. Especially in regard of metrics like BLEU, because our Freud interpretations are custom. Their exact formulation must be novel to each model.

In any case, the first observation of metrics will be treated as a baseline.

Our final observation is expected to see the improvement in each metric. This is because of the "fine-tuning" that each model will go through based on the portion of our Freud dataset. Given the fact that we have mixed metric families, i.e. BLEU vs BERT score, it's hard to predict which model will perform better.

# Metrics

## Bilingual Evaluation Understudy (BLEU)
This is a cumulative representation of precision in `N-gram` overlaps between expected and actual output.

In [26]:
def calculate_bleu(the_device, the_model, the_tokenizer, test_df):
  bleu_scores = []
  for _, a_row in test_df.iterrows():
    input_text = a_row["dream_description"]
    reference = [word_tokenize(a_row["dream_interpretation"])] # preserves punctuation tokens
    inputs = the_tokenizer(input_text, return_tensors="pt", truncation=True, max_length=MAX_TEXT_INPUT_LENGTH).to(the_device)
    smoothing = SmoothingFunction().method1 # keeps meaningful score when output is too short, i.e. not a single 2-gram matche exists
    with torch.no_grad(): # memory usage, inference speed, and accidental gradien updates are improved by disabling gradient computation
      output_ids = the_model.generate(**inputs, max_length=MAX_TEXT_OUTPUT_LENGTH)
      generated_text = the_tokenizer.decode(output_ids[0], skip_special_tokens=True)
      candidate = word_tokenize(generated_text)
      bleu = sentence_bleu(reference, candidate, smoothing_function=smoothing)
      bleu_scores.append(bleu)
    
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu

## Recall-Oriented Understudy for Gisting Evaluation (ROUGE)

In [6]:
# code

## Perplexity

In [7]:
# code

## BERT score

In [8]:
# code

# Models Init

In [13]:
current_device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

## Flan-T5-Small

In [19]:
t5_tokenizer = T5Tokenizer.from_pretrained(FLAN_T5_SMALL_NAME)
t5_model = T5ForConditionalGeneration.from_pretrained(FLAN_T5_SMALL_NAME)
t5_model.to(current_device)
print(f"{FLAN_T5_SMALL_NAME} model loaded on {current_device}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


google/flan-t5-small model loaded on cpu


## DistilBERT

In [31]:
# proceed here

# Baseline performance

In [12]:
baseline_df = pd.DataFrame(columns=["model_name", "bleu", "rouge", "perplexity", "bert_score"])

## Flan-T5-Small

In [29]:
t5_baseline_row = {
  "model_name": FLAN_T5_SMALL_NAME,
  "bleu": None,
  "rouge": None,
  "perplexity": None,
  "bert_score": None
}
t5_baseline_row["bleu"] = calculate_bleu(current_device, t5_model, t5_tokenizer, freud_test_df)

## DistilBERT

In [30]:
# proceed here